# Produce Plots for Publication

Use self built surface classification library `surface_classify` available at https://github.com/FelixWodaczek/surface_classify.git.

## Imports

In [ ]:
import sys
sys.path.append("../py_src")

from glob import glob
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.ticker import IndexLocator, MaxNLocator
from matplotlib import colors as mcolors
from matplotlib import cm
from matplotlib.colors import ListedColormap
from adjustText import adjust_text

import sort_neigh

from ase.io import read as ase_read
from ase.neighborlist import natural_cutoffs, NeighborList
from dscribe.descriptors import LMBTR, SOAP

## Standard Values

In [ ]:
simu_type = 'rh'
target_dir = "../test_data/230118_finalres_newlocalstruct_preclass/"
pd_target_dir = target_dir+'pd'
target_dir = target_dir + simu_type
n_particles = 1577
n_rhod = 15


rcut=5.2
nmax=4
lmax=3
sigma=1.
gamma_kernel=1.

## Figure 1, Available Sites on Nanoparticle

Analyse the available sites in a nanoparticle containing only Rh.

### Define Target

In [ ]:
only_cu_dir = target_dir + "/cunanoparticle"
only_cu_path = only_cu_dir + "/cusingle.lammpstrj"

In [ ]:
rh_classifier = sort_neigh.NeighbourClassifier(
    local_structures_path=os.path.abspath("../src/localstructures_final_mc"),
    non_class_max=14
)
rh_classifier.load_identifiers(
    rcut=rcut, nmax=nmax, lmax=lmax, 
    sigma=sigma, gamma_kernel=gamma_kernel,
)

### Run Analysis via Classifier
Use the surface classifier to determine what sites are identified at which position.

In [ ]:
full_particle = ase_read(only_cu_path)
at_pos = full_particle.get_positions()

In [ ]:
from ase.visualize import view
mode="class_all"

cut_off = natural_cutoffs(full_particle, mult=.98)# mult=0.98)
neighbour_list = NeighborList(cut_off, bothways=True, self_interaction=False)
neighbour_list.update(full_particle)

cu_cat_counter = np.zeros(shape=(rh_classifier.n_classes), dtype=np.int32)
categories = np.zeros((len(full_particle),), dtype=np.int32)
neighbours = np.zeros((len(full_particle),), dtype=np.int32)

ind_soaps = np.zeros((len(full_particle), rh_classifier.descr.get_number_of_features()))
for index in range(len(full_particle)):
    neighbour_indices, trash = neighbour_list.get_neighbors(index)
    neighbour_indices = np.append(np.array([index]), neighbour_indices, axis=0)
    neighbour_particle = full_particle[neighbour_indices]
    
    # Make center atom Rh
    neighbour_particle.symbols[:] = 'Cu'
    neighbour_particle.symbols[0] = 'Rh'
    
    ind_soaps[index] = rh_classifier.descr.create(neighbour_particle, positions=[0])
    n_neigh, class_id = rh_classifier.classify(neighbour_particle, mode=mode, ensure_position=False)

    cu_cat_counter[class_id] += 1
    neighbours[index] = int(n_neigh)
    categories[index] = int(class_id)
    if int(class_id) == 8:
        print(neighbour_particle)
        print(neighbour_particle.get_chemical_symbols())

In [ ]:
for ii in range(len(cu_cat_counter)):
    print(rh_classifier.id_to_cat(ii), ': %u'%cu_cat_counter[ii])

### Unsupervised Machine Learning
Use unsupervised ML to perform the same task and compare results afterwards

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans, DBSCAN, Birch
from dscribe.descriptors import SOAP

ml_classifier = sort_neigh.USMLClassifier()

train_on_particle = False
if train_on_particle:
    n_clust = ml_classifier.train_on_particle(
        full_particle,
        soap_species=["Cu"], dim_red=PCA(n_components=4), 
        clusterer=Birch(n_clusters=10),
        rcut=rcut, nmax=nmax, lmax=lmax, sigma=sigma
    )
else:
    n_clust = ml_classifier._train_on_data(
        ind_soaps,
        dim_red=PCA(n_components=2), clusterer=Birch(n_clusters=8)
    )
    ml_classifier.descr = rh_classifier.descr

# soaps = ml_classifier.descr.create(full_particle)
reduced = ml_classifier.dim_red.transform(ind_soaps)

### Plotting
Create view of nanoparticle and reduced mapping for Paper

In [ ]:
### Renaming Dict for better labels:
pretty_label = True
if pretty_label:
    prettylabel_dict = {
            "12": "12 neighbours",
            "11": "11 neighbours",
            "3_111_ad_atom": "(111) adatom",
            "4_111_ad_atom_pair": "(111) adatom pair",
            "4_100_ad_atom": "(100) adatom",
            "5_211_ad_atom": "(211) adatom" ,
            "5_110_ad_atom": "(110) adatom",
            "5_111_terrace": "(111) terrace",
            "6_100-110_interface": "(100)-(110)\ninterface",
            "6_100_terrace": "(100) terrace",
            "7_110": "(110)",
            "7_211": "(211)",
            "8_100": "(100)",
            "8_111_vacant_site": "(111) vacancy",
            "9_111": "(111)",
            "9_invalid": "9 subsurface",
            "9_invalid_2": "9 subsurface 2",
    }

In [ ]:
def found_labels(categories, labels:list=None):
    if labels is None:
        labels = np.arange(0, np.max(categories)+1).tolist()

    ret_labels = []
    for ii_label, label in enumerate(labels):
        if ii_label in categories:
            ret_labels.append(label)
    
    return ret_labels
        
def norm_categories(a_categories, labels=None):
    c_range = np.arange(np.max(a_categories), np.min(a_categories)-2, -1)
    if labels is not None and len(c_range)-1 != len(labels):
        raise ValueError("Range of argument 'counter' needs to be same length as argument 'labels' if given.")
        
    ret_cats = a_categories.copy()
    
    if labels is not None:
        ret_labels = labels.copy()
    
    reduce_at_this = False
    for value_index, value in enumerate(c_range):
        reduce_at_next = value not in ret_cats
        
        if reduce_at_this:
            largereq_than_last = ret_cats > value
            ret_cats[largereq_than_last] -= 1
            if labels is not None:
                ret_labels.pop(len(c_range)-value_index-1)

        reduce_at_this = reduce_at_next
        
    ret_cats -= np.min(ret_cats)
    return ret_cats, ret_labels
        
def cmap_from_categories(colors, or_map_name="tab20", n_orcolors=20):
    c_range = np.max(colors)-np.min(colors)+1
    normed_colors = (colors-np.min(colors))/(c_range-1)
    normed_colors *= (c_range-1)/(c_range)
    normed_colors += 1./(2*c_range)

    c_ticks = np.linspace(0, 1, c_range*2+1, endpoint=True)[1::2]

    tab20 = cm.get_cmap(or_map_name, 256)
    color_range = np.linspace(0, c_range/float(n_orcolors), 500, endpoint=False)
    cmap = ListedColormap(tab20(color_range))
    
    return normed_colors, cmap, c_ticks


In [ ]:
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)

cond = np.s_[...] # neighbours < 17

colors = categories[cond].copy()
# colors = n_clust

c_labels = np.arange(np.min(colors), np.max(colors)+1)
c_labels = [rh_classifier.id_to_cat(c_label) for c_label in c_labels]

c_labels = [rh_classifier.id_to_cat(id) for id in range(rh_classifier.n_classes)]
c_labels = found_labels(colors, c_labels)

order_by_neighbors = True
if order_by_neighbors:
    c_labels.append(c_labels[0])
    c_labels.append(c_labels[1])
    c_labels.pop(0)
    c_labels.pop(0)
    
# cont_values, c_labels = norm_categories(colors, labels=c_labels)
# cat_normed_colors, cat_cmap, cat_cticks = cmap_from_categories(cont_values, or_map_name='tab10', n_orcolors=10)
# label_color_dict = {c_label: color for c_label, color in zip(c_labels, cat_cticks)}

cat_clabels = c_labels.copy()
cat_cmap = cm.get_cmap('tab10', len(c_labels))
cat_norm = mcolors.Normalize(vmin=-0.5, vmax=len(c_labels)-0.5)
label_color_dict = {c_labels[ii_label]: cat_cmap(cat_norm(ii_label)) for ii_label in range(len(c_labels))}
label_color_dict['Non-surface'] = label_color_dict['12']

plot_colors = [label_color_dict[rh_classifier.id_to_cat(col)] for col in colors]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

sc = ax.scatter(
    at_pos[cond, 0], at_pos[cond, 1], at_pos[cond, 2], c=plot_colors, alpha=1,
    s=800, edgecolors="k"
)

if pretty_label:
    plot_labels = [
        prettylabel_dict[c_label] if c_label in prettylabel_dict.keys() else c_label for c_label in c_labels
    ]
else:
    plot_labels = c_labels
    
ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

# bar = fig.colorbar(sc, cmap=cat_cmap)
bar = fig.colorbar(cm.ScalarMappable(norm=cat_norm, cmap=cat_cmap))
bar.set_ticks(np.arange(len(c_labels)))
bar.set_ticklabels(plot_labels)
ax.set_axis_off()

plt.tight_layout()
# fig.savefig("threed_particle.pdf", format='pdf')
plt.show()

In [ ]:
def get_soaps_from_classifier(classifier):
    soaps_from_classifier = []
    labels = []

    for key in classifier.identification_dict.keys():
        entry = classifier.identification_dict[key]
        if entry is not None:
            soaps_from_classifier.append(entry["soap_descr"][:, 0, :])
            labels.append(entry["id"])

    buff = soaps_from_classifier[0].copy()
    for ii_soap in range(1, len(soaps_from_classifier)):
        buff = np.append(buff, soaps_from_classifier[ii_soap], axis=0)

    soaps_from_classifier = buff.copy()
    del buff

    buff = []
    for label in labels:
        for entry in label:
            buff.append(entry)

    labels=buff
    return soaps_from_classifier, labels

soaps_from_classifier, labels = get_soaps_from_classifier(rh_classifier)
if pretty_label:
    p_labels = [
        prettylabel_dict[blabel] if blabel in prettylabel_dict.keys() else blabel for blabel in labels
    ]
print(labels)
print(soaps_from_classifier.shape)
soap_prediction = ml_classifier.dim_red.transform(soaps_from_classifier)

In [ ]:
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)

compare_soaps = True

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("PCA Map of Unsupervised Regression")
clust_labels = np.arange(np.min(n_clust), np.max(n_clust)+1)
cmap = cm.get_cmap('tab20', np.max(n_clust)+1)

sc = ax.scatter(reduced[:, 0], reduced[:, 1], c=n_clust, cmap=cmap, vmin=np.min(n_clust)-0.5, vmax=np.max(n_clust)+0.5)

if compare_soaps:
    ax.scatter(soap_prediction[:, 0], soap_prediction[:, 1], c="k")

    for ii_label, label in enumerate(p_labels):  
        ax.annotate(label, soap_prediction[ii_label, 0:2])
    
cbar = fig.colorbar(sc)
cbar.set_ticks(clust_labels)
cbar.set_ticklabels(clust_labels)
plt.tight_layout()
# fig.savefig("unsupervised_ml_pca.pdf", format='pdf')
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

compare_soaps = True
fontsize = 12

def base_pca_particle(fig, ax,
    particle_positions, particle_colors, # Particle and its colors
    pca_points, pca_colors, 
    pca_dict_vals, dict_labels, dict_prettylabels,
    label_color_dict, 
    ms, fontsize, **kwargs
):
    sc = ax.scatter(
        pca_points[:, 0], pca_points[:, 1], 
        c=pca_colors, s=ms, rasterized=True
    )

    if kwargs.get("compare_soaps", True):
        plot_cols = []
        for ii_label, label in enumerate(dict_labels):
            try:
                plot_cols.append(label_color_dict[label])
            except:
                plot_cols.append("grey")

        ax.scatter(pca_dict_vals[:, 0], pca_dict_vals[:, 1], c=plot_cols, s=ms*kwargs.get('ms_mult', 1.3), edgecolors='k')

        texts = []
        for ii_label, label in enumerate(labels):
            texts.append(ax.text(
                pca_dict_vals[ii_label, 0], pca_dict_vals[ii_label, 1], 
                dict_prettylabels[ii_label], ha='right', fontsize=fontsize
            ))

        adjust_text(
            texts, pca_dict_vals[:, 0], pca_dict_vals[:, 1], ax=ax, 
            expand_text=kwargs.get('expand_text', (1.11, 1.25)), 
            expand_points=kwargs.get('expand_points', (1.4, 1.7)),
            force_text=kwargs.get('force_text', (0.01, 0.25)), 
            force_points=kwargs.get('force_points', (1.21, 1.21)),
            only_move={'text':'xy'}, 
            arrowprops=kwargs.get('arrowprops', dict(arrowstyle='-', lw=2, color='k', alpha=.5))
        )

    ax.tick_params(axis='x', labelsize=fontsize)
    ax.tick_params(axis='y', labelsize=fontsize)
    # ax.set_axis_off()

    axins = ax.inset_axes(bounds=kwargs.get('p_bounds', [0.05, 0.372, 0.5, 0.5]), projection='3d')
    axins.scatter(
        particle_positions[:, 0], particle_positions[:, 1], particle_positions[:, 2], c=particle_colors, alpha=kwargs.get("p_alpha", 1),
        s=kwargs.get('p_ssize', 200), edgecolors="k", rasterized=True
    )
    axins.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    axins.set_axis_off()
    x_lims = axins.get_xlim()
    x_span = x_lims[1] - x_lims[0]
    x_limmult = kwargs.get("p_xlimmult", -0.15)
    x_lims = (x_lims[0]-(x_span*x_limmult/2), x_lims[1]+(x_span*x_limmult/2))
    axins.set_xlim(x_lims)
    axins.set_ylim(x_lims)
    axins.set_zlim(x_lims)

    return fig, ax, axins

def make_pca_particle(fig, ax,
    reduced, plot_colors, # reduced particle and its colors
    soap_prediction, labels, p_labels, # reduced dicts and its colors
    label_color_dict,
    ms, fontsize, **kwargs
):
    sc = ax.scatter(
        reduced[:, 0], reduced[:, 1], 
        c=plot_colors, s=ms
    )

    if compare_soaps:
        plot_cols = []
        for ii_label, label in enumerate(labels):
            try:
                plot_cols.append(label_color_dict[label])
            except:
                plot_cols.append("grey")
        ax.scatter(soap_prediction[:, 0], soap_prediction[:, 1], c=plot_cols, s=ms*kwargs.get('ms_mult', 1.3), edgecolors='k')

        texts = []
        for ii_label, label in enumerate(labels):
            texts.append(ax.text(
                soap_prediction[ii_label, 0], soap_prediction[ii_label, 1], 
                p_labels[ii_label], ha='right', fontsize=fontsize
            ))
        adjust_text(
            texts, soap_prediction[:, 0], soap_prediction[:, 1], ax=ax, 
            expand_text=kwargs.get('expand_text', (1.11, 1.25)), 
            expand_points=kwargs.get('expand_points', (1.4, 1.7)),
            force_text=kwargs.get('force_text', (0.01, 0.25)), 
            force_points=kwargs.get('force_points', (1.21, 1.21)),
            only_move={'text':'xy'}, 
            arrowprops=kwargs.get('arrowprops', dict(arrowstyle='-', lw=2, color='k', alpha=.5))
        )

    # ax.set_xlim([-7, 21])

    ax.tick_params(axis='x', labelsize=fontsize)
    ax.tick_params(axis='y', labelsize=fontsize)
    # ax.set_axis_off()

    axins = ax.inset_axes(bounds=[0.05, 0.372, 0.5, 0.5], projection='3d')
    axins.scatter(
        at_pos[cond, 0], at_pos[cond, 1], at_pos[cond, 2], c=plot_colors, alpha=1,
        s=kwargs.get('p_ssize', 200), edgecolors="k"
    )
    axins.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    axins.set_axis_off()
    x_lims = axins.get_xlim()
    x_lims = (x_lims[0]*1.35, x_lims[1]*0.9)
    axins.set_xlim(x_lims)
    axins.set_ylim(x_lims)
    axins.set_zlim(x_lims)

    return fig, ax, axins

fig, ax = plt.subplots(1, 1, figsize=(9, 6))
if False:
    fig, ax, axin = make_pca_particle(fig, ax,
        reduced, plot_colors, # reduced particle and its colors
        soap_prediction, labels, p_labels, # reduced dicts and its colors
        label_color_dict,
        ms=70, fontsize=fontsize
    )
else:
    fig, ax, axin = base_pca_particle(fig, ax,
        particle_positions=at_pos, particle_colors=plot_colors,
        pca_points=reduced, pca_colors=plot_colors,
        pca_dict_vals=soap_prediction, dict_labels=labels, dict_prettylabels=p_labels,
        label_color_dict=label_color_dict, ms=70, fontsize=fontsize
    )
        
bar = fig.colorbar(cm.ScalarMappable(norm=cat_norm, cmap=cat_cmap))
bar.set_ticks(np.arange(len(c_labels)))
bar.set_ticklabels(plot_labels, fontsize=fontsize)

fig.savefig("fig1b_classification_pca.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

## Figure 2, Rh Positions over Time
After defining all the available positions in the nanoparticle, analyse where the Rh sit in each timestep.

### Run Evaluation on Files
Pre-existing evaluation is loaded from .txt files.

In [ ]:
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)

def load_all(target_dir):
    target_folders = [
        target_dir+"/mc",
        target_dir+"/mcmd"
    ]

    results_dict = {}
    n_cats = 0

    bulkify = True # Sum up 11 and 12 neighbours to "bulk"

    for target_folder in target_folders:
        for target_file in glob(target_folder+"/*.lammpstrj"):
            target_file = os.path.abspath(target_file)
            only_file = os.path.basename(target_file).split(".")[0]
            out_dir = os.path.join(os.path.dirname(target_file), only_file+"_soap_sorted_counts.txt")
            save_txt_path = os.path.join(os.path.dirname(target_file), only_file+"_soap_sorted_counts.txt")
            
            dir_name = save_txt_path.split("/")[-2]
            cur_key = '_'.join([dir_name, only_file])
            results_dict[cur_key] = {}
            
            sorted_counts, timesteps, sorted_cats = sort_neigh.NeighbourSort.load_sort_cat(save_txt_path)
            
            # print(dir_name)
            # sort_neigh.NeighbourSort.plot_dist(sorted_cats, sorted_counts)
            
            if bulkify:
                bulk_args = []
                for ii_scat, scat in enumerate(sorted_cats):
                    try:
                        if int(scat) >= 10 and int(scat) < 99:
                            bulk_args.append(ii_scat)
                    except:
                        pass
                bulk_args = np.array(bulk_args)
                bulk_slicer = np.s_[:, bulk_args]
                sorted_counts = np.append(
                    sorted_counts, np.sum(
                        sorted_counts[bulk_slicer], axis=-1
                    )[:, np.newaxis], axis=-1
                )
                sorted_cats.append('Non-surface')
                sorted_counts[bulk_slicer] = 0

            results_dict[cur_key]["sorted_counts"] = sorted_counts
            results_dict[cur_key]["timesteps"] = timesteps
            results_dict[cur_key]["sorted_cats"] = sorted_cats
            n_cats = max(n_cats, len(sorted_cats))
    
    return results_dict, n_cats

results_dict, n_cats = load_all(target_dir=target_dir)
pd_results_dict, pd_ncats = load_all(target_dir=pd_target_dir)

### Plot Raw Results

In [ ]:
def reshape_res_dict(results_dict):
    block_size = 1000
    md_counts = np.zeros((3 , n_cats), dtype=np.float64)
    md_errs = md_counts.copy()
    mcmd_counts = np.zeros((3 , n_cats), dtype=np.float64)
    mcmd_errs = mcmd_counts.copy()

    counter = 0
    for key, subdict in sorted(results_dict.items()):
        if not counter: # For first dictionary
            sorted_cats = subdict["sorted_cats"]
        if not "mcmd" in key:
            md_counts[counter%3, :] = np.sum(subdict["sorted_counts"], axis=0)
            md_counts[counter%3, :] = md_counts[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            md_counts[counter%3, :] *= 100

            md_errs[counter%3, :] = sort_neigh.NeighbourSort.block_average(subdict["sorted_counts"], block_size=block_size)        
            md_errs[counter%3, :] = md_errs[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            md_errs[counter%3, :] *= 100
        else:
            mcmd_counts[counter%3, :] = np.sum(subdict["sorted_counts"], axis=0)
            mcmd_counts[counter%3, :] = mcmd_counts[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            mcmd_counts[counter%3, :] *= 100
            
            mcmd_errs[counter%3, :] = sort_neigh.NeighbourSort.block_average(subdict["sorted_counts"], block_size=block_size)        
            mcmd_errs[counter%3, :] = mcmd_errs[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            mcmd_errs[counter%3, :] *= 100
        counter+=1

    return md_counts, md_errs, mcmd_counts, mcmd_errs, sorted_cats

md_counts, md_errs, mcmd_counts, mcmd_errs, sorted_cats = reshape_res_dict(results_dict=results_dict)
md_notzero = np.logical_not(np.logical_and.reduce(md_counts==0, axis=0))
mcmd_notzero = np.logical_not(np.logical_and.reduce(mcmd_counts==0, axis=0))

pd_md_counts, pd_md_errs, pd_mcmd_counts, pd_mcmd_errs, pd_sorted_cats = reshape_res_dict(results_dict=pd_results_dict)
pd_md_notzero = np.logical_not(np.logical_and.reduce(pd_md_counts==0, axis=0))
pd_mcmd_notzero = np.logical_not(np.logical_and.reduce(pd_mcmd_counts==0, axis=0))

## Plot Total Results

### Barplot

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

fontsize=20

temperatures = ['400', '500', '600']

fig, axis = plt.subplots(figsize=(12,8))
widths = np.array([-0.3, 0, 0.3])
alphas = [0.2, 0.5, 1]

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if md_notzero[ii_cat]:
        draw_cats.append(cat)

total_nonzero = np.sum(md_notzero.astype(np.int8))
x_locs = np.arange(total_nonzero)
draw_bars = md_counts[:, md_notzero]

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(label_color_dict[draw_cat])
    except:
        draw_colors.append('gray')


legend_patches = []
for ii_bars in range(md_counts.shape[0]):
    # Draw categories that are on nanoparticle in colour
    axis.bar(
        x_locs+widths[ii_bars], draw_bars[ii_bars, :], width=0.3,
        color=draw_colors, align='center', alpha=alphas[ii_bars]
    )
    
    # Make annotations with values
    annotation_cond = draw_bars[ii_bars, :] != 0
    annotations = draw_bars[ii_bars, :][annotation_cond]
    for ll_ann, annotation in enumerate(annotations):
        axis.annotate(
            "%.4f"%annotation,
            [x_locs[annotation_cond][ll_ann]+widths[ii_bars], annotation*1.1], 
            ha='center', rotation=45, fontsize=fontsize-10
        )

    cur_patch = patches.Patch(
        color='k', label=temperatures[ii_bars]+" K", alpha=alphas[ii_bars]
    )
    legend_patches.append(cur_patch)

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

axis.set_yscale("log")
axis.set_xticks(x_locs, plot_labels, fontsize=fontsize-7)
axis.tick_params(axis='y', labelsize=fontsize-7)

axis.set_xlabel('site type', fontsize=fontsize)
axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
axis.set_ylim([axis.get_ylim()[0], 400])
# axis.autoscale(tight=True)
axis.legend(handles=legend_patches, fontsize=fontsize, loc='upper left')

plt.tight_layout()
# fig.savefig("barplot_mc.pdf", format="pdf")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

fontsize=20

temperatures = ['400', '500', '600']

fig, axis = plt.subplots(figsize=(12,8))
widths = np.array([-0.3, 0, 0.3])
alphas = [0.2, 0.5, 1]

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if mcmd_notzero[ii_cat]:
        draw_cats.append(cat)

total_nonzero = np.sum(mcmd_notzero.astype(np.int8))
x_locs = np.arange(total_nonzero)
draw_bars = mcmd_counts[:, mcmd_notzero]

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(label_color_dict[draw_cat])
    except:
        draw_colors.append('gray') 


legend_patches = []
for ii_bars in range(mcmd_counts.shape[0]):
    # Draw categories that are on nanoparticle in colour
    axis.bar(
        x_locs+widths[ii_bars], draw_bars[ii_bars, :], width=0.3,
        color=draw_colors, align='center', alpha=alphas[ii_bars]
    )
    
    # Make annotations with values
    annotation_cond = draw_bars[ii_bars, :] != 0
    annotations = draw_bars[ii_bars, :][annotation_cond]
    for ll_ann, annotation in enumerate(annotations):
        axis.annotate(
            "%.4f"%annotation,
            [x_locs[annotation_cond][ll_ann]+widths[ii_bars], annotation*1.1], 
            ha='center', rotation=45, fontsize=fontsize-10
        )

    cur_patch = patches.Patch(
        color='k', label=temperatures[ii_bars]+" K", alpha=alphas[ii_bars]
    )
    legend_patches.append(cur_patch)

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

axis.set_yscale("log")
axis.set_xticks(x_locs, plot_labels, fontsize=fontsize-7)
axis.tick_params(axis='y', labelsize=fontsize-7)

axis.set_xlabel('site type', fontsize=fontsize)
axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
axis.set_ylim([axis.get_ylim()[0], 400])
# axis.autoscale(tight=True)
axis.legend(handles=legend_patches, fontsize=fontsize, loc='upper left')

plt.tight_layout()
# fig.savefig("barplot_mcmd.pdf", format="pdf")
plt.show()

### Broken Axis

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

markers = ['o', 'v', 's', '*', 'X', 'D', 'p', 'h']
fontsize = 15

md_lines = md_counts[:, mcmd_notzero]
md_draw_errs = md_errs[:, mcmd_notzero]
total_nonzero = np.sum(md_notzero.astype(np.int8))
x_range = np.arange(3)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6,5))
fig.subplots_adjust(hspace=0.05)

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if md_notzero[ii_cat]:
        draw_cats.append(cat)

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(label_color_dict[draw_cat])
    except:
        draw_colors.append('gray')

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

for axis in axes:
    for ii_line in range(-1, -total_nonzero-1, -1):
        axis.errorbar(
            x_range, md_lines[:, ii_line], yerr=md_draw_errs[:, ii_line],
            color=draw_colors[ii_line], marker=markers[ii_line],
            label=plot_labels[ii_line]    
        )
        axis.fill_between(
            x_range, 
            md_lines[:, ii_line]-md_draw_errs[:, ii_line], 
            md_lines[:, ii_line]+md_draw_errs[:, ii_line],
            color=draw_colors[ii_line],
            alpha=0.5
        )

axes[0].set_ylim([90, 100])
axes[1].set_ylim([2, 5])
axes[2].set_ylim([-0.015, 0.065])

# Format ylabels
offset = axes[0].get_ylim()[0]+(0.1*axes[0].get_ylim()[0])
axes[0].yaxis.set_major_locator(MaxNLocator(3))
axes[1].yaxis.set_major_locator(MaxNLocator(2))
axes[2].yaxis.set_major_locator(MaxNLocator(3))
axes[0].minorticks_on()
axes[1].minorticks_on()
axes[2].minorticks_on()

# Interruption
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
axes[0].plot([0, 1], [0, 0], transform=axes[0].transAxes, **kwargs)
axes[1].plot([0, 1], [1, 1], transform=axes[1].transAxes, **kwargs)
axes[1].plot([0, 1], [0, 0], transform=axes[1].transAxes, **kwargs)
axes[2].plot([0, 1], [1, 1], transform=axes[2].transAxes, **kwargs)

# set xlabels
axes[0].spines.bottom.set_visible(False)
axes[1].spines.top.set_visible(False)
axes[1].spines.bottom.set_visible(False)
axes[2].spines.top.set_visible(False)
axes[0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[1].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].set_xticks(x_range)
axes[2].set_xticklabels(temperatures, fontsize=fontsize*0.8)

# axes[0].grid()
# axes[1].grid()
# axes[2].grid()

# axis.set_yscale('log')

axes[0].set_title("Title Suggestions Welcome, mc %s"%simu_type, fontsize=fontsize)
axes[1].legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
axes[2].set_xlabel('Temperature [K]', fontsize=fontsize)
axes[1].set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)

# fig.savefig("lines_brokenax_mc_%s.pdf"%simu_type, format='pdf')
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

markers = ['o', 'v', 's', '*', 'X', 'D', 'p', 'h']
fontsize = 15

mcmd_lines = mcmd_counts[:, mcmd_notzero]
mcmd_draw_errs = mcmd_errs[:, mcmd_notzero]
total_nonzero = np.sum(mcmd_notzero.astype(np.int8))
x_range = np.arange(3)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6,5))
fig.subplots_adjust(hspace=0.05)

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if mcmd_notzero[ii_cat]:
        draw_cats.append(cat)

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(label_color_dict[draw_cat])
    except:
        draw_colors.append('gray')

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

for axis in axes:
    for ii_line in range(-1, -total_nonzero-1, -1):
        axis.errorbar(
            x_range, mcmd_lines[:, ii_line], yerr=mcmd_draw_errs[:, ii_line],
            color=draw_colors[ii_line], marker=markers[ii_line],
            label=plot_labels[ii_line]    
        )
        axis.fill_between(
            x_range, 
            mcmd_lines[:, ii_line]-mcmd_draw_errs[:, ii_line], 
            mcmd_lines[:, ii_line]+mcmd_draw_errs[:, ii_line],
            color=draw_colors[ii_line],
            alpha=0.5
        )

axes[0].set_ylim([96.4, 100])
axes[1].set_ylim([0.46, 1.53])
axes[2].set_ylim([-0.015, 0.038])

# Format ylabels
offset = axes[0].get_ylim()[0]+(0.1*axes[0].get_ylim()[0])
axes[0].yaxis.set_major_locator(MaxNLocator(3))
axes[1].yaxis.set_major_locator(MaxNLocator(2))
axes[2].yaxis.set_major_locator(MaxNLocator(3))
axes[0].minorticks_on()
axes[1].minorticks_on()
axes[2].minorticks_on()

# Interruption
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
axes[0].plot([0, 1], [0, 0], transform=axes[0].transAxes, **kwargs)
axes[1].plot([0, 1], [1, 1], transform=axes[1].transAxes, **kwargs)
axes[1].plot([0, 1], [0, 0], transform=axes[1].transAxes, **kwargs)
axes[2].plot([0, 1], [1, 1], transform=axes[2].transAxes, **kwargs)

# set xlabels
axes[0].spines.bottom.set_visible(False)
axes[1].spines.top.set_visible(False)
axes[1].spines.bottom.set_visible(False)
axes[2].spines.top.set_visible(False)
axes[0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[1].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].set_xticks(x_range)
axes[2].set_xticklabels(temperatures, fontsize=fontsize*0.8)

# axes[0].grid()
# axes[1].grid()
# axes[2].grid()

# axis.set_yscale('log')

axes[0].set_title("Title Suggestions Welcome, mcmd %s"%simu_type, fontsize=fontsize)
axes[1].legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
axes[2].set_xlabel('Temperature [K]', fontsize=fontsize)
axes[1].set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)

# fig.savefig("lines_brokenax_mcmd_%s.pdf"%simu_type, format='pdf')
plt.show()

### Logplot All Data

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

from matplotlib.lines import Line2D

def draw_rhpd(axis, rh_lines, rh_errs, pd_lines, pd_errs, all_not_zero, **kwargs):
    markers = kwargs.get('markers', ['o', 'v', 's', '*', 'X', 'D', 'p', 'h', "^"])
    fontsize = kwargs.get('fontsize', 15)
    ms = kwargs.get('ms', 10)
    lw = kwargs.get('lw', 3)

    total_nonzero = np.sum(all_not_zero.astype(np.int8))
    x_range = np.arange(3)

    draw_cats = []
    for ii_cat, cat in enumerate(sorted_cats):
        if all_not_zero[ii_cat]:
            draw_cats.append(cat)
            # print(cat, md_counts[:, ii_cat])

    draw_colors = []
    for draw_cat in draw_cats:
        try:
            draw_colors.append(label_color_dict[draw_cat])
        except:
            draw_colors.append('gray')

    if pretty_label:
        plot_labels = [
            prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
        ]
    else:
        plot_labels = draw_cats

    legend_elements = []
    for ii_line in range(total_nonzero):
        nz_tf = rh_lines[:, ii_line] != 0
        axis.errorbar(
            x_range[nz_tf], rh_lines[:, ii_line][nz_tf], yerr=rh_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker=markers[ii_line%len(markers)],
            markersize=ms, linewidth=lw,
            label=plot_labels[ii_line]
        )
        nz_tf = pd_lines[:, ii_line] != 0
        axis.errorbar(
            x_range[nz_tf], pd_lines[:, ii_line][nz_tf], yerr=pd_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker=markers[ii_line%len(markers)], 
            linestyle='--', markersize=ms, fillstyle='none', linewidth=lw,
            label=plot_labels[ii_line]
        )
        legend_elements.append(Line2D(
            [0], [0],
            linestyle="",
            color=draw_colors[ii_line], fillstyle='left',
            marker=markers[ii_line%len(markers)], markersize=ms,
            lw=lw, label=plot_labels[ii_line]))

    axis.set_yscale('log')

    axis.set_xticks(x_range)
    axis.set_xticklabels(temperatures, fontsize=fontsize*0.8)

    # axis.set_ylim((1e0, 1.3e2))

    axis.set_title(kwargs.get('title', 'nd'), fontsize=fontsize)
    # axis.legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
    axis.set_xlabel('Temperature [K]', fontsize=fontsize)
    # axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
    return axis, legend_elements

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

%matplotlib inline

from matplotlib.ticker import IndexLocator, MaxNLocator
from matplotlib import colors

all_not_zero = np.logical_or(pd_md_notzero, md_notzero)

mcmd_all_not_zero = np.logical_or(pd_mcmd_notzero, mcmd_notzero)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 6), gridspec_kw={'height_ratios': [3, 1]})

for ii_row in range(2):
    draw_rhpd(
        axes[ii_row, 0], 
        md_counts[:, all_not_zero], md_errs[:, all_not_zero],
        pd_md_counts[:, all_not_zero], pd_md_errs[:, all_not_zero],
        all_not_zero,
        title='mc'
    )
    draw_rhpd(
        axes[ii_row, 1], 
        mcmd_counts[:, mcmd_all_not_zero], mcmd_errs[:, mcmd_all_not_zero],
        pd_mcmd_counts[:, mcmd_all_not_zero], pd_mcmd_errs[:, mcmd_all_not_zero],
        mcmd_all_not_zero,
        title='mcmd'
    )

axes[0, 0].set_ylim([5e-1, 1.15e2])
axes[0, 1].set_ylim([5e-1, 1.15e2])
axes[1, 0].set_ylim([1e-4, 1e-1])
axes[1, 1].set_ylim([1e-4, 1e-1])

# Remove everything in center
axes[0, 0].set_xlabel('', visible=False)
axes[0, 1].set_xlabel('', visible=False)
axes[1, 0].set_title('', visible=False)
axes[1, 1].set_title('', visible=False)

axes[0, 0].set_xticklabels('', visible=False)
axes[0, 1].set_xticklabels('', visible=False)

for ii_row in range(2):
    for ii_col in range(2):
        axes[ii_row, ii_col].grid(axis='y', which='major', visible=True, color='k', alpha=0.8)
        axes[ii_row, ii_col].grid(axis='y', which='minor', visible=True, alpha=0.6, linestyle='--')

plt.tight_layout()
# fig.savefig("lines_allog_mcmcmd.pdf", format='pdf')
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

fs = 15
lw = 3
ms = 10
markers = ['o', 'v', 'D', '*', '>', 's', 'X', 'p', "^"]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(3, 6), gridspec_kw={'height_ratios': [3, 1]})

draw_rhcounts = mcmd_counts.copy()
draw_rherrs = mcmd_errs.copy()
draw_pdcounts = pd_mcmd_counts.copy()
draw_pderrs = pd_mcmd_errs.copy()

bulk_cats = ["9_invalid", "9_invalid_2"]
for ii_cat, cat in enumerate(sorted_cats):
    if cat in bulk_cats:
        print(cat)
        draw_rhcounts[:, -1] += draw_rhcounts[:, ii_cat]
        draw_rherrs[:, -1] += draw_rherrs[:, ii_cat]
        draw_pdcounts[:, -1] += draw_pdcounts[:, ii_cat]
        draw_pderrs[:, -1] += draw_pderrs[:, ii_cat]

        draw_rhcounts[:, ii_cat] = 0
        draw_rherrs[:, ii_cat] = 0
        draw_pdcounts[:, ii_cat] = 0
        draw_pderrs[:, ii_cat] = 0

        mcmd_all_not_zero[ii_cat] = False

# sorted_cats[-1] = "non-surface"

for ii_row in range(2):
    ax, legend_elements = draw_rhpd(
        axes[ii_row],
        mcmd_counts[:, mcmd_all_not_zero], mcmd_errs[:, mcmd_all_not_zero],
        pd_mcmd_counts[:, mcmd_all_not_zero], pd_mcmd_errs[:, mcmd_all_not_zero],
        mcmd_all_not_zero,
        title='mcmd', fontsize=fs, lw=lw, ms=ms,
        markers=markers
    )

axes[0].set_ylim([5e-1, 1.15e2])
axes[1].set_ylim([1e-4, 1e-1])

# Remove everything in center
axes[0].set_xlabel('', visible=False)
axes[0].set_title('', visible=False)
axes[1].set_title('', visible=False)
axes[0].set_xticklabels('', visible=False)

for ii_row in range(2):
    axes[ii_row].grid(axis='y', which='major', visible=True, color='k', alpha=0.8)
    axes[ii_row].grid(axis='y', which='minor', visible=True, alpha=0.6, linestyle='--')

fig.supylabel(r'Percentage of Total Configurations [%]', fontsize=fs)

sort_legend_inds = [8, 7, 5, 6, 4, 3, 2, 1, 0]
pretty_legend_elements = [legend_elements[sort_ind] for sort_ind in sort_legend_inds]

plt.tight_layout()
axes[0].legend(
    handles=[
        Line2D([0], [0], marker='o', label="Rh", color='k', linewidth=lw-1.3, markersize=ms),
        Line2D([0], [0], linestyle='--', marker='o', label="Pd", color='k', fillstyle='none', linewidth=lw-1.3, markersize=ms)
    ],
    fontsize=fs-3, loc=(1.07, 0.92), ncols=2
)
axes[1].legend(handles=pretty_legend_elements, fontsize=fs-3, loc=(1.07, 2.1))

fig.savefig("fig2_lines_allog_mcmd.pdf", format='pdf', bbox_inches = "tight")
plt.show()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

from matplotlib.lines import Line2D

def draw_rhpd_twoax(axes, rh_lines, rh_errs, pd_lines, pd_errs, all_not_zero, label_color_dict, **kwargs):
    fontsize = kwargs.get('fontsize', 12)
    ms = kwargs.get('ms', 10)
    lw = kwargs.get('lw', 3)

    nz_slicer = np.s_[:, all_not_zero]
    rh_lines = rh_lines[nz_slicer]
    rh_errs = rh_errs[nz_slicer]
    pd_lines = pd_lines[nz_slicer]
    pd_errs = pd_errs[nz_slicer]

    total_nonzero = np.sum(all_not_zero.astype(np.int8))
    x_range = np.arange(3)

    draw_cats = []
    for ii_cat, cat in enumerate(sorted_cats):
        if all_not_zero[ii_cat]:
            draw_cats.append(cat)
            # print(cat, md_counts[:, ii_cat])

    draw_colors = []
    for draw_cat in draw_cats:
        try:
            draw_colors.append(label_color_dict[draw_cat])
        except:
            draw_colors.append('gray')

    if pretty_label:
        plot_labels = [
            prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
        ]
    else:
        plot_labels = draw_cats

    legend_elements = []
    for ii_line in range(total_nonzero):
        nz_tf = rh_lines[:, ii_line] != 0
        axes[0].errorbar(
            x_range[nz_tf], rh_lines[:, ii_line][nz_tf], yerr=rh_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker='o',
            markersize=ms, linewidth=lw,
            label=plot_labels[ii_line]
        )
        nz_tf = pd_lines[:, ii_line] != 0
        axes[1].errorbar(
            x_range[nz_tf], pd_lines[:, ii_line][nz_tf], yerr=pd_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker='o', markersize=ms, linewidth=lw,
            label=plot_labels[ii_line]
        )
        legend_elements.append(Line2D(
            [0], [0],
            linestyle="",
            color=draw_colors[ii_line], fillstyle='left',
            marker='o', markersize=ms,
            lw=lw, label=plot_labels[ii_line]))

    for axis in axes:
        axis.set_yscale('log')

        axis.set_xticks(x_range)
        axis.set_xticklabels(temperatures, fontsize=fontsize*0.8)

    return axes, legend_elements

In [ ]:
draw_rhcounts = mcmd_counts.copy()
draw_rherrs = mcmd_errs.copy()
draw_pdcounts = pd_mcmd_counts.copy()
draw_pderrs = pd_mcmd_errs.copy()

mcmd_all_not_zero = np.logical_or(mcmd_notzero, pd_mcmd_notzero)
total_nonzero = np.sum(mcmd_all_not_zero.astype(np.int8))
x_range = np.arange(3)

# add categories to bulk that are not initially in there
bulk_cats = ["9_invalid", "9_invalid_2"]
for ii_cat, cat in enumerate(sorted_cats):
    if cat in bulk_cats:
        print("Bulkified: ", cat)
        draw_rhcounts[:, -1] += draw_rhcounts[:, ii_cat]
        draw_rherrs[:, -1] += draw_rherrs[:, ii_cat]
        draw_pdcounts[:, -1] += draw_pdcounts[:, ii_cat]
        draw_pderrs[:, -1] += draw_pderrs[:, ii_cat]

        draw_rhcounts[:, ii_cat] = 0
        draw_rherrs[:, ii_cat] = 0
        draw_pdcounts[:, ii_cat] = 0
        draw_pderrs[:, ii_cat] = 0

        mcmd_all_not_zero[ii_cat] = False

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if mcmd_all_not_zero[ii_cat]:
        draw_cats.append(cat)
        # print(draw_pdcounts[:, ii_cat], cat)

data_cat_cmap = cm.get_cmap('tab10', len(draw_cats))
data_cat_norm = mcolors.Normalize(vmin=-0.5, vmax=len(draw_cats)-0.5)
data_label_color_dict = {draw_cats[ii_label]: data_cat_cmap(data_cat_norm(ii_label)) for ii_label in range(len(draw_cats))}
data_label_color_dict['12'] = data_label_color_dict['Non-surface']
data_label_color_dict['11'] = data_label_color_dict['Non-surface']
data_label_color_dict['10'] = data_label_color_dict['Non-surface']
for bcat in bulk_cats:
    data_label_color_dict[bcat] = data_label_color_dict['Non-surface']

In [ ]:
class VerboseSorter(sort_neigh.NeighbourSort):
    def get_cats_dimred(self, dim_red, cutoff_mult=0.9, last_n=15, **kwargs):
        cat_ids = np.zeros((self.timesteps*last_n,), dtype=np.int8)
        pcas = np.zeros((self.timesteps*last_n, 2), dtype=np.float32)

        init_particle = self.particle_trajectory[0]
        particle_len = len(init_particle)
        particle_range = (particle_len - np.arange(last_n)[::-1])-1

        cut_off = natural_cutoffs(init_particle, mult=cutoff_mult)
        neighbour_list = NeighborList(cut_off, bothways=True, self_interaction=False)
        neighbour_list.update(init_particle)

        for step in self.progressbar(range(self.timesteps), "At Timestep:", size=40):
            cur_particle = self.particle_trajectory[step]
            neighbour_list.update(cur_particle)

            for ii_ind, index in enumerate(particle_range):
                neighbour_indices, dists = neighbour_list.get_neighbors(index)
                neighbour_indices = np.append(np.array([index]), neighbour_indices, axis=0)

                neighbour_particle = cur_particle[neighbour_indices]
                neighbour_particle.symbols[:] = 'Cu'
                neighbour_particle.symbols[0] = 'Rh'
                n_neighbours = len(neighbour_particle) - 1
                
                n_neigh, class_id = self.classifier.classify(neighbour_particle, **kwargs)
                soaps = self.classifier.descr.create(neighbour_particle, positions=[0])

                cat_ids[(step*last_n)+ii_ind] = class_id
                pcas[(step*last_n)+ii_ind, :] = dim_red.transform(soaps)[..., :2]

        return cat_ids, pcas

new_soaps = False
new_sorter = VerboseSorter(
    local_structures_path=os.path.abspath("../src/localstructures_final_mcmd_pd"),
    non_class_max=14, rcut=rcut, nmax=nmax, lmax=lmax,
    sigma=sigma, gamma_kernel=gamma_kernel,
)
traj_path = "../test_data/221229_saafinal/pd/mcmd/600.lammpstrj"
new_sorter.load_particle(traj_path)
if new_soaps:
    example_catids, example_pca = new_sorter.get_cats_dimred(dim_red=ml_classifier.dim_red, last_n=n_rhod, mode='pre_group')
    np.save("example_catids.npy", example_catids)
    np.save("example_pca.npy", example_pca)
else:
    example_catids, example_pca = np.load("example_catids.npy"), np.load("example_pca.npy")



In [ ]:
example_colors = [data_label_color_dict[new_sorter.classifier.id_to_cat(catid)] for catid in example_catids]

example_ts = 3412
example_pos = new_sorter.particle_trajectory[example_ts].get_positions()
cu_col = 'peru'

# Go through particle, if a Rh is found get classification from 
example_pcolors, example_sizes = [], []
rh_counter = 0
for ii_symb, symb in enumerate(new_sorter.particle_trajectory[example_ts].get_chemical_symbols()):
    if symb == 'Rh':
        example_pcolors.append(example_colors[(example_ts*n_rhod)+rh_counter])
        example_sizes.append(125)
        rh_counter += 1
    else:
        example_pcolors.append(cu_col)
        example_sizes.append(0.5)

example_soaps_from_classifier, example_labels = get_soaps_from_classifier(new_sorter.classifier)
if pretty_label:
    example_plabels = [
        prettylabel_dict[blabel] if blabel in prettylabel_dict.keys() else blabel for blabel in example_labels
    ]
example_soap_prediction = ml_classifier.dim_red.transform(example_soaps_from_classifier)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
if True:
    fontsize = 12
    font = {'family': 'serif', 'size': fontsize}
    plt.rc('font', **font)
    plt.rc('text', usetex=True)
    plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

fig = plt.figure(figsize=(11, 6))
fontsize = 12
ms = 7

marg = 0.05 # margin between figures

rsw = 0.4 # width of right side
lsw = 1 - rsw - marg # width of left side

lmarg = 0.025 # margin between logplots
lth = 0.6 # total height of logplots
ltw = (lsw - (3*marg))/3
lm = 3 # ratio of upper to lower logplot
luh = ((lth-marg-lmarg)/(lm+1))*lm # height of upper logplot
llh = (lth-marg-lmarg)/(lm+1) # height of lower logplot

rate_axes = []
rate_axes.append(fig.add_axes((0, llh+lmarg, ltw, luh))) # First logplot top
rate_axes.append(fig.add_axes((0, 0, ltw, llh))) # First logplot bottom
rate_axes.append(fig.add_axes((ltw+marg, llh+lmarg, ltw, luh))) # Second logplot top
rate_axes.append(fig.add_axes((ltw+marg, 0, ltw, llh))) # Second logplot bottom

# here is where the colorbar goes
cax = fig.add_axes(((ltw+marg)*2, lmarg, ltw/2, lth-lmarg*4))

soap_axes = []
soap_axes.append(fig.add_axes((lsw+marg, ((1-marg)/2)+marg, rsw, (1-marg)/2)))
soap_axes.append(fig.add_axes((lsw+marg, 0, rsw, (1-marg)/2)))

# Plotting
## Logplots
### Draw
plt_ax, legend_elements = draw_rhpd_twoax([rate_axes[0], rate_axes[2]], 
    rh_lines=draw_rhcounts, rh_errs=draw_rherrs, 
    pd_lines=draw_pdcounts, pd_errs=draw_pderrs, 
    all_not_zero=mcmd_all_not_zero, 
    label_color_dict=data_label_color_dict, 
    fontsize=fontsize, ms=ms
)
rate_axes[0] = plt_ax[0]
rate_axes[2] = plt_ax[1]
plt_ax, legend_elements = draw_rhpd_twoax([rate_axes[1], rate_axes[3]], 
    rh_lines=draw_rhcounts, rh_errs=draw_rherrs, 
    pd_lines=draw_pdcounts, pd_errs=draw_pderrs, 
    all_not_zero=mcmd_all_not_zero, 
    label_color_dict=data_label_color_dict, 
    fontsize=fontsize, ms=ms
)
rate_axes[1] = plt_ax[0]
rate_axes[3] = plt_ax[1]

### Beautification of logplots
for large_ind in range(0, 4, 2):
    rate_axes[large_ind].set_ylim([5e-1, 1.15e2])
    rate_axes[large_ind].set_xlabel('', visible=False)
    rate_axes[large_ind].set_title('', visible=False)
    rate_axes[large_ind].set_xticklabels('', visible=False)

for small_ind in range(1, 4, 2):
    rate_axes[small_ind].set_ylim([1e-4, 1e-1])
    rate_axes[small_ind].set_title('', visible=False)
    rate_axes[small_ind].set_xlabel('Temperature (K)', fontsize=fontsize)


for rax in rate_axes:
    rax.grid(axis='y', which='major', visible=True, color='k', alpha=0.8)
    rax.grid(axis='y', which='minor', visible=True, alpha=0.6, linestyle='--')

for right_ind in range(2, 4):
    rate_axes[right_ind].set_yticklabels([], visible=False)

rate_axes[1].set_ylabel(r'Percentage of Total Configurations (\%)', fontsize=fontsize, loc="bottom")
# rate_axes[0].yaxis.set_label_coords(-0.1,-0.3)
rate_axes[0].set_ylabel('')

pretty_cats = []
for draw_cat in draw_cats:
    pretty_cats.append(prettylabel_dict.get(draw_cat, draw_cat))

bar = fig.colorbar(cm.ScalarMappable(norm=data_cat_norm, cmap=data_cat_cmap), cax=cax)
bar.set_ticks(np.arange(len(draw_cats)))
bar.set_ticklabels(pretty_cats, fontsize=fontsize)

if False:
    fig, soap_axes[0], axin = make_pca_particle(fig, soap_axes[0],
        reduced, [data_label_color_dict[rh_classifier.id_to_cat(col)] for col in categories[cond]], # reduced particle and its colors
        soap_prediction, labels, p_labels, # reduced dicts and its colors
        data_label_color_dict,
        ms=50, fontsize=fontsize-3,
        p_ssize=130, expand_text=(0.8, 0.9), expand_points=(1.4, 1.4), force_text=(0.6, 0.6), force_points=(0.4, 0.4)
    )
else:
    red_colors = [data_label_color_dict[rh_classifier.id_to_cat(col)] for col in categories]
    fig, soap_axes[0], axin = base_pca_particle(fig, soap_axes[0],
        particle_positions=at_pos, particle_colors=red_colors,
        pca_points=reduced, pca_colors=red_colors,
        pca_dict_vals=soap_prediction, dict_labels=labels, dict_prettylabels=p_labels,
        label_color_dict=data_label_color_dict, 
        ms=50, fontsize=fontsize-3,
        p_ssize=130, p_xlimmult=-0.25,
        p_bounds=[0.0, 0.472, 0.5, 0.5],
        expand_text=(0.5, 0.9), expand_points=(1.6, 1.6), force_text=(0.6, 0.6), force_points=(0.4, 0.4)
    )
    fig, soap_axes[1], axin_1 = base_pca_particle(fig, soap_axes[1],
        particle_positions=example_pos, particle_colors=example_pcolors, # p_alpha=example_palphas,
        pca_points=example_pca, pca_colors=example_colors,
        pca_dict_vals=example_soap_prediction, dict_labels=example_labels, dict_prettylabels=example_plabels,
        label_color_dict=data_label_color_dict,
        ms=50, fontsize=fontsize-3,
        p_bounds=[0.0, 0.472, 0.5, 0.5],
        p_ssize=example_sizes, p_xlimmult=-0.25,
        expand_text=(0.8, 0.9), expand_points=(1.6, 1.6), force_text=(0.6, 0.6), force_points=(0.4, 0.4)
    )

soap_axes[0].set_axis_off()
soap_axes[1].set_axis_off()

fig.savefig("fig1_classification_pca.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

## Fig 4b

In [ ]:
Palladium = np.array([1.970249e-02, 4.653179e-03, 6.216751e-03, 9.130835e-09, 1.421196e-07, 3.180858e-02,
            3.873732e-06, 2.207631e-04, 7.291755e-07, 1.873654e-06, 1.275488e-06, 2.568031e-06, 1.791322e-09])
Rhodium = np.array([2.947998e-01, 4.076129e+00, 2.213683e+00, 1.360925e-02, 4.343079e-03, 1.279555e+01,
           1.363255e+00, 1.273051e-01, 1.756916e-03, 3.369158e-01, 3.047968e-02, 1.655088e-03, 9.261856e-04])

assert np.all(Palladium<Rhodium), "Palladium more active than Rhodium at some site"

plot_rh = Rhodium-Palladium

labels = ['(111) adatom', '(100) adatom', '(111) adatom pair', '(110) adatom', '(111) terrace', '(211) adatom',
'(100) terrace', '(100)-(110) interface', '(110)', '(211)', '(100)', '(111) vacancy', '(111)']
# Get old labels for label_color_dict
reg_labels = [list(prettylabel_dict.keys())[list(prettylabel_dict.values()).index(label)] for label in labels]
draw_colors = []
for reg_label in reg_labels:
        try:
            draw_colors.append(label_color_dict[reg_label])
        except:
            draw_colors.append('gray')


width = 0.7       # the width of the bars: can also be len(x) sequence
fontsize=14

fig, ax = plt.subplots(figsize=(6,4))
plt.rcParams["hatch.linewidth"] = 2
ax.bar(np.arange(len(labels)), Palladium, width, label='Palladium', color='tab:blue') # , hatch=r"//", edgecolor="w")
ax.bar(np.arange(len(labels)), plot_rh, width, bottom=Palladium, label='Rhodium', color='tab:orange')

plt.yscale("log")
ax.set_ylim([1e-10, 1e3])
ax.set_ylabel(r'Rate (s$^{-1}$)', fontsize=fontsize)
ax.legend(fontsize=fontsize-4)

ax.set_xticks(np.arange(len(labels)), labels=labels, rotation=-35, rotation_mode="anchor", ha='left', fontsize=fontsize-4)

plt.tight_layout()
fig.savefig("fig3b_rate.pdf", format='pdf')
plt.show()